In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np
from sklearn.metrics import mean_squared_error as mse

In [2]:
data=pd.read_csv('./data/washed_train_data.csv',encoding='gbk')
data = data.fillna(0)#.drop(["property_other","invest_other","edu_other"],axis=1)
y = data["happiness"]               
x = data.drop(['happiness','Unnamed: 0'],axis=1)
# Split data into train and test setsPython
# X_train, X_test, Y_train, Y_test = train_test_split(x, y,test_size=0.2,random_state=123)

# scaling data
#scaler = preprocessing.StandardScaler().fit(X_train)
#x_scaled = scaler.transform(x)
#X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# full data train
scaler = preprocessing.StandardScaler().fit(x)
X_train_scaled =X_test_scaled = scaler.transform(x)
Y_train=Y_test=y

In [3]:
# LGBMRegressor

from lightgbm.sklearn import LGBMRegressor

LGBMmodel = LGBMRegressor(n_jobs=-1,learning_rate=0.051,
                      n_estimators=400,
                      num_leaves=11,
                      reg_alpha=2.0, 
                      reg_lambda=2.1,
                      min_child_samples=6,
                      min_split_gain=0.5,
                      colsample_bytree=0.2
                     )

LGBMmodel.fit(X_train_scaled,Y_train)
mse(Y_test,LGBMmodel.predict(X_test_scaled))

0.344449471736143

In [4]:
#CatBoostRegressor
from catboost import Pool, CatBoostRegressor


CBRmodel = CatBoostRegressor(colsample_bylevel=0.1,thread_count=6,silent=True,iterations=800, 
                          depth=5, 
                          learning_rate=0.051, 
                          loss_function='RMSE',
                          l2_leaf_reg = 3)


CBRmodel.fit(X_train_scaled,Y_train)
mse(Y_test,CBRmodel.predict(X_test_scaled))

0.35294683007262456

In [5]:
#gbdt
from sklearn.ensemble import GradientBoostingRegressor


GBDTmodel = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.051, loss='ls', max_depth=4, max_features=10,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=600, presort='auto', random_state=3,
             subsample=0.98, verbose=0, warm_start=False)

GBDTmodel.fit(X_train_scaled,Y_train)
mse(Y_test,GBDTmodel.predict(X_test_scaled))

0.2762206983756079

In [6]:
from xgboost.sklearn import XGBRegressor
#xgboost
XGBmodel = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.1,
       colsample_bytree=0.971, gamma=0.11, learning_rate=0.069, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=499,
       n_jobs=-1, nthread=50, objective='reg:linear', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)

XGBmodel.fit(X_train_scaled, Y_train)
mse(XGBmodel.predict(X_test_scaled),Y_test)

/home/x/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/x/.local/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.33960263198642837

In [7]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge

res = np.c_[LGBMmodel.predict(X_test_scaled),
            XGBmodel.predict(X_test_scaled),
            GBDTmodel.predict(X_test_scaled),
            XGBmodel.predict(X_test_scaled)]
lr = Ridge(fit_intercept=False, alpha=75)
lr.fit(res,Y_test)
lr.coef_


array([-0.15094163, -0.24200641,  1.63756683, -0.24200641])

In [8]:
mse(Y_test,lr.predict(res))


0.24886135445235363

In [9]:
res_1 = np.c_[LGBMmodel.predict(X_train_scaled),
            XGBmodel.predict(X_train_scaled),
            GBDTmodel.predict(X_train_scaled),
            XGBmodel.predict(X_train_scaled)]
mse(Y_train,lr.predict(res_1))

0.24886135445235363

In [10]:
def predict(data):
    res_1 = np.c_[LGBMmodel.predict(data),
            XGBmodel.predict(data),
            GBDTmodel.predict(data),
            XGBmodel.predict(data)]
    return lr.predict(res_1)

In [12]:
test_data=pd.read_csv('./data/washed_test_data.csv',encoding='gbk')
test_data = test_data.fillna(0)
test_data_scaled = scaler.transform(test_data.drop(['Unnamed: 0'],axis=1))
final =predict(test_data_scaled).tolist()
res_id=[i for i in range(8001,10968+1)]
res_csv=pd.DataFrame([final],columns=res_id,index=['happiness']).T
res_csv.to_csv("/home/x/文档/git/AliTianChi/Happiness/res/res4.csv")